#### Check at doi.org if the article citing the dataset is a preprint

DOI endpoint: GET https://doi.org/ (in your header: Accept application/json)



e.g. GET https://doi.org/10.5194/acp-2018-513 will return metadata about the article and should include the 'type' (subtype?) of publication

pass list of publications DOIs that have cited datasets to DOI.org to check if they are pre-prints

In [1]:
import requests
import pandas as pd
import time

In [30]:
# doi_url = 'https://doi.org/10.1016/j.agee.2014.04.009' # posted content
#10.5285/5a711904-ef42-46a3-9f47-3f0d6b231f65'
#'https://doi.org/10.5194/acp-2018-513'
 
publicationDOI = '10.5194/acp-2018-513'
doi_url = 'https://doi.org/' + publicationDOI
headers={"Accept": "application/json"}

r = requests.get(doi_url, headers = {"Accept": "application/json"})
print(r.status_code)
r.json()

200


{'indexed': {'date-parts': [[2022, 4, 5]],
  'date-time': '2022-04-05T20:18:33Z',
  'timestamp': 1649189913271},
 'posted': {'date-parts': [[2018, 7, 19]]},
 'group-title': 'Aerosols/Field Measurements/Troposphere/Physics (physical properties and processes)',
 'reference-count': 0,
 'publisher': 'Copernicus GmbH',
 'license': [{'start': {'date-parts': [[2018, 7, 19]],
    'date-time': '2018-07-19T00:00:00Z',
    'timestamp': 1531958400000},
   'content-version': 'unspecified',
   'delay-in-days': 0,
   'URL': 'https://creativecommons.org/licenses/by/4.0/'}],
 'funder': [{'DOI': '10.13039/100010418',
   'name': 'Defence Science and Technology Laboratory',
   'doi-asserted-by': 'publisher',
   'award': ['R119928']}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'abstract': '<jats:p>Abstract. Primary biological aerosol particles (PBAP) are an abundant subset of atmospheric aerosol particles which comprise viruses, bacteria, fungal spores, pollen, and fragments such 

In [29]:
r.headers

{'date': 'Mon, 13 Jun 2022 14:16:58 GMT', 'content-type': 'application/json', 'content-length': '2150', 'vary': 'Accept, Accept-Encoding', 'link': '<http://dx.doi.org/10.5194/acp-2018-513>; rel="canonical", <https://www.atmos-chem-phys-discuss.net/acp-2018-513/acp-2018-513.pdf>; version="vor"; rel="item", <https://creativecommons.org/licenses/by/4.0/>; rel="license", <http://orcid.org/0000-0003-0719-3258>; title="Elizabeth Forde"; rel="author", <http://orcid.org/0000-0002-4968-6088>; title="Martin Gallagher"; rel="author", <http://orcid.org/0000-0003-4433-7310>; title="Ian Crawford"; rel="author", <http://orcid.org/0000-0001-6950-4870>; title="Paul Kaye"; rel="author", <http://orcid.org/0000-0001-8247-9649>; title="David Topping"; rel="author"', 'access-control-expose-headers': 'Link', 'access-control-allow-headers': 'X-Requested-With, Accept, Accept-Encoding, Accept-Charset, Accept-Language, Accept-Ranges, Cache-Control', 'access-control-allow-origin': '*', 'content-encoding': 'gzip',

In [7]:
r.json()

{'indexed': {'date-parts': [[2022, 4, 5]],
  'date-time': '2022-04-05T20:18:33Z',
  'timestamp': 1649189913271},
 'posted': {'date-parts': [[2018, 7, 19]]},
 'group-title': 'Aerosols/Field Measurements/Troposphere/Physics (physical properties and processes)',
 'reference-count': 0,
 'publisher': 'Copernicus GmbH',
 'license': [{'start': {'date-parts': [[2018, 7, 19]],
    'date-time': '2018-07-19T00:00:00Z',
    'timestamp': 1531958400000},
   'content-version': 'unspecified',
   'delay-in-days': 0,
   'URL': 'https://creativecommons.org/licenses/by/4.0/'}],
 'funder': [{'DOI': '10.13039/100010418',
   'name': 'Defence Science and Technology Laboratory',
   'doi-asserted-by': 'publisher',
   'award': ['R119928']}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'abstract': '<jats:p>Abstract. Primary biological aerosol particles (PBAP) are an abundant subset of atmospheric aerosol particles which comprise viruses, bacteria, fungal spores, pollen, and fragments such 

In [27]:
pubType = r.json()['type']
pubType

'posted-content'

In [6]:
r.json()['subtype']

'preprint'

In [36]:
def checkDOIpubType(publicationDOI):
    headers={"Accept": "application/json"}
    doi_url = 'https://doi.org/' + publicationDOI
    
    try:
        r = requests.get(doi_url, headers={"Accept": "application/json"}) # sometimes throws up unexpected errors
        print(r.status_code, publicationDOI)
        # sometimes the API call returns info in a format that is not JSON 
        
        pubType = r.json()['type'] # check if returned info is in json format and if there is publication type info

        if 'subtype' in r.json() == True:
            pubSubType = r.json()['subtype']
            print(pubType, ': ', publicationDOI)
        else:
            print(pubType, ': ', publicationDOI)
            pubSubType = 'None'

    except Exception as e: # if unexpected error or returned info is not in json format put pubtype as unknown to be dealt with later
        print('no json: ', publicationDOI)
        print(e)
        pubType = 'unknown'
        pubSubType = 'unknown'

    return pubType, pubSubType


publicationDOI = '10.5194/acp-2018-513'
pubType = checkDOIpubType(publicationDOI)
print(pubType)

200 10.5194/acp-2018-513
posted-content :  10.5194/acp-2018-513
('posted-content', 'None')


In [8]:
# define a function to be used on the DOI of each publication that cites datasets to check what type of publication it is
def checkDOIpubType(publicationDOI):
    
    #doi_org = 'https://doi.org/'
    #doi_url = doi_org + publicationDOI
    r = requests.get(('https://doi.org/' + publicationDOI), 
                     headers={"Accept": "application/json"})
    
    try:
        pubType = r.json()['type']
            
        if 'subtype' in r.json() == True:
            pubSubType = r.json()['subtype']
            # I should record both type and subtype
            
            print(pubType, ': ', publicationDOI)
        else:
            print(pubType, ': ', publicationDOI)
            pubSubType = 'None'
            pass 
    except:
        print('no json: ', publicationDOI)
        pubType = 'unknown'
        pubSubType = 'unknown'
          
    return pubType, pubSubType

    
    

In [9]:
#publicationDOI = '10.5194/acp-2018-513' # change this to get the DOI from the Dataframe collected from Scholex and DataCite
#publicationDOI = '10.5194/acp-19-9699-2019'
publicationDOI = '10.5445/ir/1000059774'
test = checkDOIpubType(publicationDOI)
test

no json:  10.5445/ir/1000059774


('unknown', 'unknown')

In [11]:
# get DOIs of publications that have cited datasets
# pass them to DOI.org to determine type of publication

publicationDOIList = scholex_df['pubIDs']

pubTypeList = []

for count, doi in enumerate(publicationDOIList):
    pubType = checkDOIpubType(doi)
    pubTypeList.append(pubType)
    
    time.sleep(1)
    if count % 200 == 0: # if count is a multiple of 200 wait for a bit
            time.sleep(180)

print('Done!')    
pubTypeList

journal-article :  10.1007/s00300-015-1855-0
journal-article :  10.5194/cp-17-111-2021
no json:  10.5285/e15d622c-5c7e-45e9-b127-f27def94bbe8
no json:  10.5285/42cb66b9-f515-41bc-8b9a-6dc9cb0e5624
no json:  10.5285/e15d622c-5c7e-45e9-b127-f27def94bbe8
no json:  10.5285/e15d622c-5c7e-45e9-b127-f27def94bbe8
journal-article :  10.1093/icesjms/fsab092
no json:  10.5285/1a266e7b-ef8d-4db7-8e7c-155be21d17cd
no json:  10.5285/85356871-55ee-4b92-a9e8-9b370c843936
no json:  10.5285/a5174472-204d-45d4-862c-3befc32b52ca
journal-article :  10.1093/icesjms/fsab092
no json:  10.5285/1a266e7b-ef8d-4db7-8e7c-155be21d17cd
no json:  10.5285/85356871-55ee-4b92-a9e8-9b370c843936
journal-article :  10.1093/icesjms/fsab092
no json:  10.5285/a5174472-204d-45d4-862c-3befc32b52ca
no json:  10.5285/8d60d83b-d880-4042-b344-42d0551e65be
no json:  10.5285/c8d5a7a4-e19b-497f-b7ae-56f9a2b15a56
journal-article :  10.1093/icesjms/fsab092
journal-article :  10.1093/icesjms/fsab092
no json:  10.5285/f9993ee9-085f-42cf-b

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [14]:
print(len(pubTypeList))
print(len(scholex_df['pubIDs']))

1023
1418


In [ ]:
pad = len(scholex_df['pubIDs'])
pubTypeList += [''] * (pad - len(pubTypeList)) # pad the pubTypeList so that its the same length and can be added to the scholex_df 
scholex_df['pubTypeList'] = pubTypeList

In [29]:
scholex_df.to_csv('scholex_API_response_pubType_130422.csv', index = False)

In [ ]:
scholex_df.head(20)

In [ ]:
#scholex_df.filter(like='posted-content', axis=0)

newdf = scholex_df[(scholex_df.pubTypeList == "posted")]

newdf

In [ ]:
#scholex_df = pd.read_csv('scholex_API_response_130422.csv')

In [ ]:
# For unknown pubtypes i.e. the html ones
# Send the DOI to doiRA API - 
# Determine if crossref or datacite
# Then send DOI to them
# Determine publication type from their metadata

In [33]:
doi_url = 'https://doi.org/doiRA/10.5194/cp-2017-18'
#10.17863/cam.56230' # posted content

headers = {
    "Accept": "application/json"
}

r = requests.get(doi_url, headers=headers)
print(r.status_code)
r.json()

200


[{'DOI': '10.5194/cp-2017-18', 'RA': 'Crossref'}]

In [34]:
r.json()[0]['RA']

'Crossref'

In [35]:
# ask the Datacite API what type of publication
datacite_url = 'https://api.datacite.org/dois/10.17863/cam.56230'
#datacite_url = 'https://api.test.datacite.org/dois/
# format = '?format=json'
# page = '?page=87'
# url = catalogue_url + page

headers = {
    'client-id': 'bl.nerc',
#    "Accept": "application/json"
}

r = requests.get(datacite_url, headers = headers)
print(r.status_code)
r.json()

print(r.json()['data']['attributes']['types'])
print(r.json()['data']['attributes']['types']['citeproc'])

200
{'ris': 'RPRT', 'bibtex': 'article', 'citeproc': 'article-journal', 'schemaOrg': 'ScholarlyArticle', 'resourceType': 'Article', 'resourceTypeGeneral': 'Text'}
article-journal


In [51]:
r.json()['data']['attributes']['types']

{'ris': 'RPRT',
 'bibtex': 'article',
 'citeproc': 'article-journal',
 'schemaOrg': 'ScholarlyArticle',
 'resourceType': 'Article',
 'resourceTypeGeneral': 'Text'}

In [6]:
# ask the crossref API what type of publication

doi_url = 'https://api.crossref.org/works/10.17863/cam.47435'
#  10.1002/ece3.2461

headers = {
    "Accept": "application/json"
}

r = requests.get(doi_url, headers=headers)
print(r.status_code)
r.json()

print(r.json()['message']['type'])
r.json()['message']


404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# determine publication type when DOI.org fails

# get the unknown rows from scholex_df pubtype column and the pubDOI
for pubType, pubDOI in (scholex_df['pubTypeList'],scholex_df['pubTypeList']):
    
    if pubType == 'unknown':

        # determine if crossref or datacite supplies the DOI
        doi_url = 'https://doi.org/doiRA/' + pubDOI

        r = requests.get(doi_url, headers={"Accept": "application/json"})
        DOIregistry = r.json()['RA']
        print(DOIregistry)

        # query the crossref or datacite API
        if DOIregistry == 'DataCite':
            # ask the Datacite API what type of publication
            datacite_url = 'https://api.datacite.org/dois/10.17863/cam.56230'
            r = requests.get(datacite_url, headers = {'client-id': 'bl.nerc'})
            print(r.status_code)

            newPubType = r.json()['data']['attributes']['types'],
            #unknownPubType = r.json()['data']['attributes']['types']['citeproc']

            print(newPubType)
            

        elif DOIregistry == 'Crossref':
            doi_url = 'https://api.crossref.org/works/'  + pubDOI
            r = requests.get(doi_url, headers={"Accept": "application/json"})
            print(r.status_code)

            newPubType = {'type': r.json()['message']['type'], 'subtype':r.json()['message']['subtype']}
            print(newPubType)

        else:
            print('Unknown DOI registry')
        
    else:
        newPubType = pubType



